#### Русскоязычный текст (119 слов):  
*Мария затопила **печь** и принялась замешивать тесто. Ей всегда нравилось **печь** пироги и **бейглы**, но, увы, это дело редко получалось хорошо. В прошлый раз **стекло** старой миски лопнуло и дало **течь**, и свежее тесто начало незаметно **течь** на скатерть, а потом и вовсе **стекло** на пол. В другой раз в печи было так жарко, что сгорел и пирог, и **жаркое** к ужину, пока Мария пила малиновый чай и любовалась на **жаркое** летнее солнце. **Временами** от готовки её отвлекал сосед, **что** любил жаловаться всем подряд, **что** с такими **временами** года урожай гибнет и работать в поле приходится всё больше. "Если уж выбрал для жизни **село**, то будь готов работать с самого утра и пока солнце не **село**" - отвечала ему Мария.*  

Выделенные жирным слова представлют сложность для POS-теггера, потому что имеют грамматические омонимы. Некоторые слова различить довольно просто, если учитывать контекст вокруг них, а в некоторых случаях это не поможет, например: *пока солнце не село*. Сочетания типа сущ + не + сущ вполне возможны в русском языке. Также в тексте присутствует несловарное слово *бейглы*, которое парсерам придётся распознавать по аналогии (а сначала хорошо или не очень хорошо её подобрать).  
#### Англоязычный текст (117 слов):  
*State your name and date of birth for the **record**, please.  
We need you to **record** more messages because your voice **sounds** more natural than **mine**.  
They must drive **fast** to get to that magic **well** in time.  
Several natural molecules perform a **fast** internal conversion.  
Somebody told me he works at that **mine**.  
Self-defense is also a constitutional **right** and fighting for it is the **right** thing to do.  
I do not feel **well** now so I need to **lie** down and rest for a bit.  
We are able to detect any **lie** you say, dear.  
All I did was **light** a **match** and then everything exploded!  
She picked my suit to **match** her wedding dress.*  

Это уже не связный текст, а отдельные предложения, потому что я не смогла придумать так же хорошо, как на русском. Но тут тоже есть как более простые для определения омонимы, так и сложные случаи типа *a fast internal conversion*, где сочетание *a fast* можно расценить как существительное, или омоним *well*, крайне редко встречающийся в виде существительного. Выделены только некоторые потенциально сложные слова, за счёт бедной английской морфологии и помимо специально добавленных слов в тексте много возможностей для ошибки.

In [62]:
import json
import navec
import nltk
import spacy
import re
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from nltk import word_tokenize, pos_tag
from pymorphy2 import MorphAnalyzer
from pprint import pprint
from flair.data import Sentence
from flair.models import SequenceTagger
from pymystem3 import Mystem
from sklearn.metrics import accuracy_score
m = MorphAnalyzer()
mst = Mystem()

In [2]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)

In [3]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

### Разметка текстов

In [7]:
# открыть тексты из файлов
with open('test_rus.txt', encoding='UTF-8') as f:
    test_rus = f.read()
with open('test_eng.txt', encoding='UTF-8') as f:
    test_eng = f.read()

In [6]:
print(test_rus)

Мария затопила печь и принялась замешивать тесто. Ей всегда нравилось печь пироги и бейглы, но, увы, это дело редко получалось хорошо. В прошлый раз стекло старой миски лопнуло и дало течь, и свежее тесто начало незаметно течь на стол, а потом и вовсе стекло на пол. В другой раз в печи было так жарко, что сгорел и пирог, и жаркое к ужину, пока Мария пила малиновый чай и любовалась на жаркое летнее солнце. Временами от готовки её отвлекал сосед, что любил жаловаться всем подряд, что с такими временами года урожай гибнет и работать в поле приходится всё больше. "Если уж выбрал для жизни село, то будь готов работать с раннего утра и пока солнце не село" - отвечала ему Мария.


In [3]:
# разметить русский текст
golden_rus = []
for word in word_tokenize(test_rus):
    print(word)
    golden_rus.append(input())

Мария
NOUN
затопила
VERB
печь
NOUN
и
CONJ
принялась
VERB
замешивать
INFN
тесто
NOUN
.
PNKT
Ей
NPRO
всегда
ADVB
нравилось
VERB
печь
INFN
пироги
NOUN
и
CONJ
торты
NOUN
,
PNKT
но
CONJ
,
PNKT
увы
INTJ
,
PNKT
это
DONTKNOW
дело
NOUN
редко
ADVB
получалось
VERB
хорошо
ADVB
.
PNKT
В
PREP
прошлый
ADJF
раз
NOUN
стекло
NOUN
старой
ADJF
миски
NOUN
лопнуло
VERB
и
CONJ
дало
VERB
течь
NOUN
,
PNKT
и
CONJ
свежее
ADJF
тесто
NOUN
начало
VERB
незаметно
ADVB
течь
INFN
на
PREP
стол
NOUN
,
PNKT
а
CONJ
потом
ADVB
и
CONJ
вовсе
ADVB
стекло
VERB
на
PREP
пол
NOUN
.
PNKT
В
PREP
другой
ADJF
раз
NOUN
в
PREP
печи
NOUN
было
VERB
так
ADVB
жарко
ADVB
,
PNKT
что
CONJ
сгорел
VERB
и
CONJ
пирог
NOUN
,
PNKT
и
CONJ
жаркое
NOUN
к
PREP
ужину
NOUN
,
PNKT
пока
CONJ
Мария
NOUN
пила
VERB
малиновый
ADJF
чай
NOUN
и
CONJ
любовалась
VERB
на
PREP
жаркое
ADJF
летнее
ADJF
солнце
NOUN
.
PNKT
Временами
ADVB
от
PREP
готовки
NOUN
её
NPRO
отвлекал
VERB
сосед
NOUN
,
PNKT
что
RELPRO
любил
VERB
жаловаться
INFN
всем
NPRO
подряд
ADVB
,
PNKT
что
CONJ

In [61]:
print(golden_rus)

['NOUN', 'VERB', 'NOUN', 'CONJ', 'VERB', 'VERB', 'NOUN', 'PNKT', 'PRON', 'ADVB', 'VERB', 'VERB', 'NOUN', 'CONJ', 'NOUN', 'PNKT', 'CONJ', 'PNKT', 'INTJ', 'PNKT', 'PRON', 'NOUN', 'ADVB', 'VERB', 'ADVB', 'PNKT', 'PREP', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'CONJ', 'VERB', 'NOUN', 'PNKT', 'CONJ', 'ADJ', 'NOUN', 'VERB', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PNKT', 'CONJ', 'ADVB', 'CONJ', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PNKT', 'PREP', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'VERB', 'ADVB', 'ADVB', 'PNKT', 'CONJ', 'VERB', 'CONJ', 'NOUN', 'PNKT', 'CONJ', 'NOUN', 'PREP', 'NOUN', 'PNKT', 'CONJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'PREP', 'ADJ', 'ADJ', 'NOUN', 'PNKT', 'ADVB', 'PREP', 'NOUN', 'PRON', 'VERB', 'NOUN', 'PNKT', 'PRON', 'VERB', 'VERB', 'PRON', 'ADVB', 'PNKT', 'CONJ', 'PREP', 'ADJ', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'CONJ', 'VERB', 'PREP', 'NOUN', 'VERB', 'ADVB', 'ADJ', 'PNKT', 'PNKT', 'CONJ', 'PRCL', 'VERB', 'PREP', 'NOUN', 'NOUN', 'PNKT', 'CONJ', 'VERB', 'ADJ', 'VERB', 'PREP', '

In [72]:
# привести к читабельному виду
tokened = word_tokenize(test_rus)
pretty_golden_rus = []
for w, g in zip(tokened, golden_rus):
    pretty_golden_rus.append({w:g})
pprint(pretty_golden_rus[:10])

[{'Мария': 'NOUN'},
 {'затопила': 'VERB'},
 {'печь': 'NOUN'},
 {'и': 'CONJ'},
 {'принялась': 'VERB'},
 {'замешивать': 'VERB'},
 {'тесто': 'NOUN'},
 {'.': 'PNKT'},
 {'Ей': 'PRON'},
 {'всегда': 'ADVB'}]


In [7]:
print(test_eng)

State your name and date of birth for the record, please.
We need you to record more messages because your voice sounds more natural than mine.
They must drive fast to get to that magic well in time.
Several natural molecules perform a fast internal conversion.
Somebody told me he works at that mine.
Self-defense is also a constitutional right and fighting for it is the right thing to do.
I do not feel well now so I need to lie down and rest for a bit.
We are able to detect any lie you say, dear.
All I did was light a match and then everything exploded!
She picked my suit to match her wedding dress.


In [108]:
# разметить английский текст
golden_eng = []
for word in word_tokenize(test_eng):
    print(word)
    golden_eng.append(input())

State
VERB
your
PRON
name
NOUN
and
CONJ
date
NOUN
of
PREP
birth
NOUN
for
PREP
the
DET
record
NOUN
,
PUNCT
please
INTJ
.
PUNCT
We
PRON
need
VERB
you
PRON
to
PRCL
record
VERB
more
ADJ
messages
NOUN
because
CONJ
your
PRON
voice
NOUN
sounds
VERB
more
ADJ
natural
ADJ
than
CONJ
mine
PRON
.
PUNCT
They
PRON
must
VERB
drive
VERB
fast
ADV
to
PRCL
get
VERB
to
PREP
that
DET
magic
ADJ
well
NOUN
in
PREP
time
NOUN
.
PUNCT
Several
ADJ
natural
ADJ
molecules
NOUN
perform
VERB
a
DET
fast
ADJ
internal
ADJ
conversion
NOUN
.
PUNCT
Somebody
PRON
told
VERB
me
PRON
he
PRON
works
VERB
at
PREP
that
DET
mine
NOUN
.
PUNCT
Self-defense
NOUN
is
VERB
also
ADV
a
DET
constitutional
ADJ
right
NOUN
and
CONJ
fighting
VERB
for
PREP
it
PRON
is
VERB
the
DET
right
ADJ
thing
NOUN
to
PRCL
do
VERB
.
PUNCT
I
PRON
do
VERB
not
PRCL
feel
VERB
well
ADV
now
ADV
so
CONJ
I
PRON
need
VERB
to
PRCL
lie
VERB
down
ADV
and
CONJ
rest
VERB
for
PREP
a
DET
bit
NOUN
.
PUNCT
We
PRON
are
VERB
able
ADJ
to
PRCL
detect
VERB
any
DET
lie
NOUN
you
PRON
sa

In [112]:
print(golden_eng)

['VERB', 'PRON', 'NOUN', 'CONJ', 'NOUN', 'PREP', 'NOUN', 'PREP', 'DET', 'NOUN', 'PUNCT', 'INTJ', 'PUNCT', 'PRON', 'VERB', 'PRON', 'PRCL', 'VERB', 'ADJ', 'NOUN', 'CONJ', 'PRON', 'NOUN', 'VERB', 'ADV', 'ADJ', 'CONJ', 'PRON', 'PUNCT', 'PRON', 'VERB', 'VERB', 'ADV', 'PRCL', 'VERB', 'PREP', 'DET', 'ADJ', 'NOUN', 'PREP', 'NOUN', 'PUNCT', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'DET', 'ADJ', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'PRON', 'PRON', 'VERB', 'PREP', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'VERB', 'ADV', 'DET', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'PREP', 'PRON', 'VERB', 'DET', 'ADJ', 'NOUN', 'PRCL', 'VERB', 'PUNCT', 'PRON', 'VERB', 'PRCL', 'VERB', 'ADV', 'ADV', 'CONJ', 'PRON', 'VERB', 'PRCL', 'VERB', 'ADV', 'CONJ', 'VERB', 'PREP', 'DET', 'NOUN', 'PUNCT', 'PRON', 'VERB', 'ADJ', 'PRCL', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'PUNCT', 'ADJ', 'PUNCT', 'PRON', 'PRON', 'VERB', 'VERB', 'VERB', 'DET', 'NOUN', 'CONJ', 'ADV', 'PRON', 'VERB', 'PUNCT', 'PRON', 'VERB', 'PRON', 'NOUN', 'PRCL', 'VERB', 'PRON', 'NOUN',

In [113]:
tokened = word_tokenize(test_eng)
pretty_golden_eng = []
for w, g in zip(tokened, golden_eng):
    pretty_golden_eng.append({w:g})
pprint(pretty_golden_eng[:10])

[{'State': 'VERB'},
 {'your': 'PRON'},
 {'name': 'NOUN'},
 {'and': 'CONJ'},
 {'date': 'NOUN'},
 {'of': 'PREP'},
 {'birth': 'NOUN'},
 {'for': 'PREP'},
 {'the': 'DET'},
 {'record': 'NOUN'}]


### Русские парсеры

In [86]:
# возвращает список тегов и читабельные словарики
def test_pym2(text):
    tags = []
    pretty = []
    for word in word_tokenize(text):
        result = m.parse(word)[0].tag.POS
        if result == None: # у пунктуации нет атрибута POS
            tags.append('PNKT') # это для accuracy
            pretty.append({word:'PNKT'}) # это посмотреть глазами
        else:
            tags.append(str(result))
            pretty.append({word:str(result)})
    return(tags, pretty)

In [87]:
# то же самое для майстема
def test_mystem(text):
    tags = []
    pretty = []
    result = mst.analyze(text)
    for res in result:
        if 'analysis' in res:
            gr = res['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            tags.append(pos)
            pretty.append({res['text']:pos})
    return(tags, pretty)

In [88]:
# то же самое для наташи
def test_natasha(text):
    tags = []
    pretty = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        pretty.append({token.text: token.pos})
        tags.append(token.pos)
    return(tags, pretty)

In [89]:
tags_pym2 = test_pym2(test_rus)
pprint(tags_pym2[1][:10])

[{'Мария': 'NOUN'},
 {'затопила': 'VERB'},
 {'печь': 'INFN'},
 {'и': 'CONJ'},
 {'принялась': 'VERB'},
 {'замешивать': 'INFN'},
 {'тесто': 'NOUN'},
 {'.': 'PNKT'},
 {'Ей': 'NPRO'},
 {'всегда': 'ADVB'}]


In [90]:
tags_mystem = test_mystem(test_rus)
pprint(tags_mystem[1][:10])

[{'Мария': 'S'},
 {'затопила': 'V'},
 {'печь': 'S'},
 {'и': 'CONJ'},
 {'принялась': 'V'},
 {'замешивать': 'V'},
 {'тесто': 'S'},
 {'Ей': 'SPRO'},
 {'всегда': 'ADVPRO'},
 {'нравилось': 'V'}]


In [91]:
tags_natasha = test_natasha(test_rus)
pprint(tags_natasha[1][:10])

[{'Мария': 'PROPN'},
 {'затопила': 'VERB'},
 {'печь': 'NOUN'},
 {'и': 'CCONJ'},
 {'принялась': 'VERB'},
 {'замешивать': 'ADJ'},
 {'тесто': 'NOUN'},
 {'.': 'PUNCT'},
 {'Ей': 'PRON'},
 {'всегда': 'ADV'}]


Проблема: у всех трёх парсеров разные теги (у наташи это формат UD) + майстем не парсит пунктуацию.  
Решение:  
1) убрать теги пунктуации везде, включая золотой стандарт;  
2) привести теги к единому стандарту с помощью правил

#### Унификация тегов

In [93]:
pym2_rule = {
    'ADJF':'ADJ',
    'ADJS':'ADJ',
    'COMP':'ADJ',
    'ADVB':'ADV',
    'NPRO':'PRON',
    'INFN':'VERB',
}

In [94]:
myst_rule = {
    'S':'NOUN',
    'A':'ADJ',
    'V':'VERB',
    'ADVPRO':'ADV',
    'PR':'PREP',
    'PART':'PRCL',
    'APRO':'PRON',
    'SPRO':'PRON',
}

In [95]:
natasha_rule = {
    'PROPN':'NOUN',
    'AUX':'VERB',
    'ADP':'PREP',
    'CCONJ':'CONJ',
    'SCONJ':'CONJ',
    'PART':'PRCL',
    'PUNCT':'PNKT',
}

In [96]:
def unify(tagset, rule):
    unified = []
    for item in tagset:
        if item in rule:
            unified.append(rule[item])
        else:
            unified.append(item)
    return unified

In [97]:
unif_pym2 = unify(tags_pym2[0], pym2_rule)
unif_myst = unify(tags_mystem[0], myst_rule)
unif_natasha = unify(tags_natasha[0], natasha_rule)

In [138]:
# удалить тег пунктуации из всех списков:
def del_pnkt(tag_list):
    while 'PNKT' in tag_list:
        tag_list.remove('PNKT')
    return tag_list

In [139]:
punkted = [golden_rus, unif_pym2, unif_natasha]
for p in punkted:
    del_pnkt(p)

In [141]:
# сверить теги
print(set(unif_pym2))
print(set(unif_myst))
print(set(unif_natasha))
print(set(golden_rus))

{'PRON', 'CONJ', 'PRCL', 'INTJ', 'VERB', 'NOUN', 'PREP', 'ADV', 'ADJ'}
{'PRON', 'CONJ', 'PRCL', 'INTJ', 'VERB', 'NOUN', 'PREP', 'ADV', 'ADJ'}
{'PRON', 'CONJ', 'PRCL', 'VERB', 'NOUN', 'DET', 'PREP', 'NUM', 'ADV', 'ADJ'}
{'PRON', 'CONJ', 'PRCL', 'INTJ', 'ADVB', 'VERB', 'NOUN', 'PREP', 'ADJ'}


#### Accuracy

In [143]:
print("Accuracy pymorphy2: %.4f" % accuracy_score(unif_pym2, golden_rus))
print("Accuracy mystem: %.4f" % accuracy_score(unif_myst, golden_rus))
print("Accuracy natasha: %.4f" % accuracy_score(unif_natasha, golden_rus))

Accuracy pymorphy2: 0.7731
Accuracy mystem: 0.8235
Accuracy natasha: 0.7983


### Английские парсеры

In [8]:
print(test_eng)

State your name and date of birth for the record, please.
We need you to record more messages because your voice sounds more natural than mine.
They must drive fast to get to that magic well in time.
Several natural molecules perform a fast internal conversion.
Somebody told me he works at that mine.
Self-defense is also a constitutional right and fighting for it is the right thing to do.
I do not feel well now so I need to lie down and rest for a bit.
We are able to detect any lie you say, dear.
All I did was light a match and then everything exploded!
She picked my suit to match her wedding dress.


In [83]:
def test_nltk(text):
    tags = []
    pretty = []
    for result in pos_tag(word_tokenize(text), tagset='universal'):
        tags.append(result[1])
        pretty.append({result[0]:result[1]})
    return(tags, pretty)

In [67]:
def test_spacy(text):
    tags = []
    pretty = []
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    for t in doc:
        tags.append(t.pos_)
        pretty.append({t.text:t.pos_})
    return(tags, pretty)

In [63]:
def test_flair(text):
    tags = []
    pretty = []
    sentence = Sentence(test_eng)
    tagger = SequenceTagger.load('upos')
    tagger.predict(sentence)
    for entity in sentence.to_dict(tag_type='pos')['entities']:
        pos = re.match('[A-Z]+', str(entity['labels'][0])).group(0)
        word = entity['text']
        tags.append(pos)
        pretty.append({word:pos})
    return(tags, pretty)

In [98]:
tags_nltk = test_nltk(test_eng)
pprint(tags_nltk[1][:10])

[{'State': 'VERB'},
 {'your': 'PRON'},
 {'name': 'NOUN'},
 {'and': 'CONJ'},
 {'date': 'NOUN'},
 {'of': 'ADP'},
 {'birth': 'NOUN'},
 {'for': 'ADP'},
 {'the': 'DET'},
 {'record': 'NOUN'}]


In [129]:
tags_spacy = test_spacy(test_eng)
pprint(tags_spacy[1][:10])

[{'State': 'VERB'},
 {'your': 'PRON'},
 {'name': 'NOUN'},
 {'and': 'CCONJ'},
 {'date': 'NOUN'},
 {'of': 'ADP'},
 {'birth': 'NOUN'},
 {'for': 'ADP'},
 {'the': 'DET'},
 {'record': 'NOUN'}]


In [103]:
tags_flair = test_flair(test_eng)
pprint(tags_flair[1][:10])

2020-10-12 15:25:46,654 loading file C:\Users\User\.flair\models\en-pos-ontonotes-v0.4.pt
[{'State': 'NOUN'},
 {'your': 'PRON'},
 {'name': 'NOUN'},
 {'and': 'CCONJ'},
 {'date': 'NOUN'},
 {'of': 'ADP'},
 {'birth': 'NOUN'},
 {'for': 'ADP'},
 {'the': 'DET'},
 {'record': 'NOUN'}]


#### Унификация тегов

In [130]:
# удалить вторую часть сложного слова,
# чтобы количество ответов парсеров совпадало
pprint(tags_spacy[1][65])
pprint(tags_spacy[1][66])
pprint(tags_spacy[1][67])
del tags_spacy[0][67]

{'Self': 'NOUN'}
{'-': 'PUNCT'}
{'defense': 'NOUN'}


In [104]:
nltk_rule = {
    '.':'PUNCT',
    'PRT':'PRCL',
    'ADP':'PREP',
}

In [105]:
spacy_rule = {
    'AUX':'VERB',
    'ADP':'PREP',
    'CCONJ':'CONJ',
    'SCONJ':'CONJ',
    'PART':'PRCL',
    'SPACE':'PUNCT',
}

In [106]:
flair_rule = {
    'AUX':'VERB',
    'ADP':'PREP',
    'CCONJ':'CONJ',
    'SCONJ':'CONJ',
    'PART':'PRCL',
}

In [131]:
unif_nltk = unify(tags_nltk[0], nltk_rule)
unif_spacy = unify(tags_spacy[0], spacy_rule)
unif_flair = unify(tags_flair[0], flair_rule)

In [136]:
def del_punct(tag_list):
    while 'PUNCT' in tag_list:
        tag_list.remove('PUNCT')
    return tag_list

In [137]:
puncted = [golden_eng, unif_nltk, unif_spacy, unif_flair]
for p in puncted:
    del_punct(p)

In [141]:
print(set(unif_nltk))
print(set(unif_spacy))
print(set(unif_flair))
print(set(golden_eng))

{'NOUN', 'VERB', 'ADJ', 'PRCL', 'DET', 'PREP', 'ADV', 'CONJ', 'PRON'}
{'NOUN', 'VERB', 'INTJ', 'ADJ', 'PRCL', 'DET', 'PREP', 'ADV', 'CONJ', 'PRON'}
{'NOUN', 'INTJ', 'VERB', 'ADJ', 'PRCL', 'DET', 'PREP', 'ADV', 'CONJ', 'PRON'}
{'NOUN', 'VERB', 'INTJ', 'ADJ', 'PRCL', 'DET', 'PREP', 'ADV', 'CONJ', 'PRON'}


#### Accuracy

In [139]:
print("Accuracy nltk: %.4f" % accuracy_score(unif_nltk, golden_eng))
print("Accuracy spacy: %.4f" % accuracy_score(unif_spacy, golden_eng))
print("Accuracy flair: %.4f" % accuracy_score(unif_flair, golden_eng))

Accuracy nltk: 0.8718
Accuracy spacy: 0.9487
Accuracy flair: 0.8974


### Итог
Итак, среди русскоязычных парсеров побеждает Mystem, а среди англоязычных - SpaCy. Но ни один из них не смог стопроцентно верно разметить мои тексты.